<a href="https://colab.research.google.com/github/leolougomes/Meu_Portfolio_Leonardo-Gomes/blob/master/Min_Max_Tic_Tac_Toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimax Algorithm - Tic Tac Toe - Game Theory

The maximin value is the highest value that the player can be sure to get without knowing the actions of the other players; equivalently, it is the lowest value the other players can force the player to receive when they know the player's action. Its formal definition is:

\begin{equation} 
v_i = \min_{a_{i}}\max_{a_{-i}}v_i(a_i,a_{-i}) = \min_{a_{i}}( \max_{a_{-i}}v_i(a_i,a_{-i}) )
\end{equation}

A simple version of the minimax algorithm deals with games such as tic-tac-toe, where each player can win, lose, or draw. If player A can win in one move, their best move is that winning move. If player B knows that one move will lead to the situation where player A can win in one move, while another move will lead to the situation where player A can, at best, draw, then player B's best move is the one leading to a draw. Late in the game, it's easy to see what the "best" move is. The Minimax algorithm helps find the best move, by working backwards from the end of the game. At each step it assumes that player A is trying to maximize the chances of A winning, while on the next turn player B is trying to minimize the chances of A winning (i.e., to maximize B's own chances of winning).

In [1]:
import math
import time
import random

# 1) Creating the Tic Tac Toe Board and Mechanics.

In [2]:
class TicTacToe():    
    
    def __init__(self):
        self.board = self.make_board()
        self.current_winner = None
        
    # In tic tac toe there is 9 spaces, We will define like that:
        
    def make_board(self):
        return [' ' for _ in range(9)]
    
    # For Visualization purpose we are gonna make a separation of this 9 space like a #.
    
    def print_board(self):
        for row in [self.board[i*3:(i+1) * 3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')
            
    # For Visualization purpose we are gonna make a board were it show us the number of the square that is empty.      
    
    def print_board_nums(self):
        # 0 | 1 | 2
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')
            
            
    # Now, we are gonna make a representation of each move. 
    # Were letter = 'X' or 'O'.
    # Square is the spaces of the board, if the space/square is empty we are gonna print the Letter, if not we are gonna return False.
    # Also in this function we are gonna chek the winner that we are gonna define by a previous function called "winner"
    
    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    # On this function we are gonna look if there is a winner first in the rows and then in the diagonals.
    # If any of the rows there are 3 letters equals than we are gonna return true.
    # Also if any of the diagonals are 3 letters equals also we return true.
    # If not, we return false, there's not a winner yet.
    
    def winner(self, square, letter):
        # check the row
        row_ind = math.floor(square / 3)
        row = self.board[row_ind*3:(row_ind+1)*3]
        # print('row', row)
        if all([s == letter for s in row]):
            return True
        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        # print('col', column)
        if all([s == letter for s in column]):
            return True
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            # print('diag1', diagonal1)
            if all([s == letter for s in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            # print('diag2', diagonal2)
            if all([s == letter for s in diagonal2]):
                return True
        return False
    

    # These 3 next functions look for the empty squares, if they exist, and then enumerates from these empty squares.
    
    def empty_squares(self):
        return ' ' in self.board

    def num_empty_squares(self):
        return self.board.count(' ')

    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == " "]

# 2) Creating Types of Players

In [3]:
class Player():
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        pass

1) Human Player:

In [4]:
class HumanPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-9): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

2) Random Computer Player.

In [5]:
# These Random Computer are just gonna make random moves.

class RandomComputerPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        square = random.choice(game.available_moves())
        return square

3) Smart Computer Player

In [6]:
class SmartComputerPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves())
        else:
            square = self.minimax(game, self.letter)['position']
        return square

    # State statement means the state of the current game
    
    def minimax(self, state, player):
        max_player = self.letter  # yourself
        other_player = 'O' if player == 'X' else 'X'

        # first we want to check if the previous move is a winner
        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player else -1 * (
                        state.num_empty_squares() + 1)}
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -math.inf}  # each score should maximize
        else:
            best = {'position': None, 'score': math.inf}  # each score should minimize
        for possible_move in state.available_moves():
            state.make_move(possible_move, player)
            sim_score = self.minimax(state, other_player)  # simulate a game after making that move

            # undo move
            state.board[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move  # this represents the move optimal next move

            if player == max_player:  # X is max player
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
        return best

3) Creating The Game

In [7]:
def play(game, x_player, o_player, print_game=True):

    if print_game:
        game.print_board_nums()

    letter = 'X'
    while game.empty_squares():
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)
        if game.make_move(square, letter):

            if print_game:
                print(letter + ' makes a move to square {}'.format(square))
                game.print_board()
                print('')

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter  # ends the loop and exits the game
            letter = 'O' if letter == 'X' else 'X'  # switches player

        time.sleep(.8)

    if print_game:
        print('It\'s a tie!')

4) PLAY!!!!!!

In [8]:
        x_player = RandomComputerPlayer('X')
        o_player = SmartComputerPlayer('O')
        t = TicTacToe()
        play(t, x_player, o_player, print_game=True)

| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |
X makes a move to square 8
|   |   |   |
|   |   |   |
|   |   | X |

O makes a move to square 4
|   |   |   |
|   | O |   |
|   |   | X |

X makes a move to square 3
|   |   |   |
| X | O |   |
|   |   | X |

O makes a move to square 0
| O |   |   |
| X | O |   |
|   |   | X |

X makes a move to square 1
| O | X |   |
| X | O |   |
|   |   | X |

O makes a move to square 2
| O | X | O |
| X | O |   |
|   |   | X |

X makes a move to square 5
| O | X | O |
| X | O | X |
|   |   | X |

O makes a move to square 6
| O | X | O |
| X | O | X |
| O |   | X |

O wins!


'O'